In [1]:
import cv2
from datetime import date
import face_recognition
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from matplotlib import pyplot as plt
import pandas as pd
import os
import mediapipe as mp
import threading

import io
from google.cloud import storage
from IPython.display import Image

import firebase_admin
from firebase_admin import auth
from firebase_admin import credentials
from firebase_admin import firestore

2023-11-24 15:39:06.871872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Funciones

In [2]:
# Funcion para actualizar la asistencia en la base de datos
def update_attendance(row, asistencias):

    if row['fecha'] == date.today() and row['id'] in asistencias:
        result = asistencias[row['id']]
    else:
        result = row['asistencia']
    return result

In [3]:
def update_participacion(row, hand_counts):
    # Asegurarse de que el nombre del alumno esté en los conteos de manos
    if row['LocalID'] in hand_counts:
        # Actualizar la participación para el alumno identificado
        participacion_actual = row['Participacion']
        nueva_participacion = participacion_actual + hand_counts[row['LocalID']]
        return nueva_participacion
    else:
        # Si el alumno no está en los conteos de manos, mantener el valor actual de participación
        return row['Participacion']

In [4]:
# Funcion para actualizar la asistencia en la base de datos
def update_attendance(row, asistencias):
    if row['LocalID'] in asistencias:
        return asistencias[row['LocalID']]
    else:
        return row['Asistencia']

In [5]:
# Función para obtener los ids de alumnes en cada clase
def id_alumnes(clave, grupo, fire):
    doc_ref = fire.collection(clave).document(grupo)
    doc = doc_ref.get()
    if doc.exists:
        correos = [element.lower() + '@tec.mx' for element in doc.to_dict()['Alumnes']]
        uids = [auth.get_user_by_email(email).uid for email in correos]
        return uids

    else:
        print("La clase no existe")
        return False

In [6]:
# Inicializar el módulo de manos de MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Función para detectar manos y obtener puntos clave en un fotograma
def detect_hand_landmarks(frame):
    # Convertir la imagen a RGB (MediaPipe requiere imágenes en formato RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Obtener las manos detectadas
    results = hands.process(frame_rgb)

    # Lista para almacenar los puntos clave de la mano
    hand_landmarks = []

    # Verificar si se detectaron manos
    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks:
            landmarks = [(landmark.x, landmark.y) for landmark in hand_landmark.landmark]
            hand_landmarks.append(landmarks)

     # Lista para almacenar las puntas de los dedos de la mano
    hand_fingertips = []

    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks:
            # Obtener la posición de la punta del dedo medio
            fingertip = (hand_landmark.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                         hand_landmark.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y)
            hand_fingertips.append(fingertip)

    return hand_landmarks, hand_fingertips


# Nube

In [ ]:
# Cloud conexión
# Llave para autenticacion del cliente
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/fernandaalcubilla/Desktop/IA-2/Reto/Interfaz/skilled-orbit-401601-1c9280b4aa9b.json'

# Inizializar el cliente
client = storage.Client()
bucket = client.get_bucket('equipo8-ia2')

# Inicialización firestore
cred = credentials.Certificate('/Users/fernandaalcubilla/Desktop/IA-2/Reto/Interfaz/usuariosia8-firebase-adminsdk-9gu37-23182c948b.json')
firebase_admin.initialize_app(cred)
fire = firestore.client()

# Modelo

In [ ]:
clave = 'TC3006C'
grupo = '2'

In [ ]:
known_face_names = id_alumnes(clave, grupo, fire)
if known_face_names:
    known_face_encodings = []
    # Base de datos para guardar asistencia y participación
    db = pd.DataFrame.from_records([(date.today(), x, '', 0) for x in known_face_names], columns =['Fecha', 'LocalID', 'Asistencia', 'Participacion'])
    for uid in known_face_names:
        blob = bucket.blob('Fotos_caras/'+uid+'.jpg')

        face = face_recognition.load_image_file(io.BytesIO(blob.download_as_bytes()))
        face_face_encoding = face_recognition.face_encodings(face)[0]
        known_face_encodings.append(face_face_encoding)

#####################################################################################################

    video_capture = cv2.VideoCapture(0) # Inicio de camara 0

    hand_raised_threshold = 0.0004 # Umbral para determinar si la mano está levantada
    asistencias = {name: 0 for name in known_face_names}
    hand_counts = {name: 0 for name in known_face_names}
    frames_count = 0

    hand_raised_previous = False
    process_this_frame = True

    while True:

        # Se agarra un solo frame del video
        ret, frame = video_capture.read()

        # Detectar manos y obtener puntos clave
        all_hand_points, hand_fingertips = detect_hand_landmarks(frame)

        # Solo se procesan algunos frames para ahorrar tiempo
        if process_this_frame:
            frames_count += 1
            # Hacer el frame 1/4 de su tamaño original
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = small_frame
            
            # Encontrar todas las caras y codigos en el frame
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # Verifica si la cara hace match con alguna del dataset
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # En caso de que tenga dudas, pone la cara mas cerca a la de alguna del dataset
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                    asistencias[name] = asistencias[name] + 1
                face_names.append(name)

        process_this_frame = not process_this_frame
        ret, frame = video_capture.read()
                
        # Cambiar de tamaño de la imagen
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)

        # VERIFICAR ROSTROS Y ASISTENCIA
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Volver a poner las imagenes en tamaño orignal antes de reducirla a 1/4 de su tamaño
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Poner la caja alrededor de las caras
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Poner los labels de las caras en la caja
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # VERIFICAR SI HAY ALGUNA MANO LEVANTADA
        for i, hand_points in enumerate(all_hand_points):
            # Obtener la posición y de la punta del dedo medio (puedes ajustar esto según tu preferencia)
            if i < len(face_names):
                tip_of_middle_finger_y = hand_points[12][1] if len(hand_points) > 15 else 0

                # Determinar si la mano está levantada
                hand_raised = tip_of_middle_finger_y < hand_raised_threshold * frame.shape[0]

                # Si la mano está levantada y no estaba levantada en el frame anterior
                name = face_names[i]
                if hand_raised and not hand_raised_previous:
                    if name in hand_counts:
                        hand_counts[name] = hand_counts[name] + 1
                        
                # Actualizar la bandera para el próximo frame
                hand_raised_previous = hand_raised


            # Dibujar un círculo en la punta del dedo medio (solo para visualización)
            cv2.circle(frame, (int(hand_points[12][0] * frame.shape[1]), int(hand_points[12][1] * frame.shape[0])),
                    5, (0, 255, 0), -1)

            # Dibujar un texto indicando si la mano está levantada
            cv2.putText(frame, f"Mano {i + 1} - Levantada: {hand_raised}", (10, 30 + i * 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0) if hand_raised else (0, 0, 255), 2)
            

        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

    valores = np.fromiter(asistencias.values(), dtype=float)
    # Dividimos todos los valores por el número de frames
    divided_values = valores / frames_count

    # Rango para convertir a cualitativo
    conditions = [
        (divided_values >= 0) & (divided_values <= .05),
        (divided_values > .05) & (divided_values <= .20),
        (divided_values > .20) & (divided_values <= .60),
        (divided_values > .60)
    ]
    choices = ['Inasistencia', 'Bajo' ,'Medio', 'Alto']
    rango_valores = np.select(conditions, choices, default='Unknown')
    asistencias = dict(zip(asistencias.keys(), rango_valores))

    # Actualizar asistencia
    db['Participacion'] = db.apply(update_participacion, axis=1, args=(hand_counts,))
    db['Asistencia'] = db.apply(update_attendance, axis=1, args=(asistencias,))


In [ ]:
db

,Fecha,LocalID,Asistencia,Participacion
0,2023-11-22,H7ry96mrFhW9Gg4YCNRM8zmAB9o2,Inasistencia,0
1,2023-11-22,Cs3yJrR6PtbfnZFRx7TL0Q3EUqv1,Medio,2
2,2023-11-22,Xl7cRHJaircpk2wZQoqLCbvfyIc2,Inasistencia,0


# Upload to bucket

In [ ]:
# Subir db de Asistencia y participación a la nube
# Cree un objeto Blob que represente el archivo parquet que desea cargar.
blob = bucket.blob('Cursos/'+clave+'/'+grupo+'/' + str(date.today()) + '.parquet')

# Sube el dataframe directamente a Google Storage como un string.
blob.upload_from_string(db.to_parquet(compression='gzip'), '.parquet')